In [1]:
import pandas as pd
from transformers import pipeline
# from datasets import load_dataset
# from datasets import Dataset
# from tqdm.auto import tqdm
# import torch

In [4]:
df = pd.read_parquet('../data/5b.cryptonews_topic_sentiment.parquet')
df.head(3)

,title,text,source_name,date,topic,topic_confidence_score,news_text_title,sentiment_finbert,score_finbert,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa,People,Organizations,Cryptocurrencies,Events
0,2021 Bitcoin Price Predictions: Is The Massive...,As the bitcoin price hovers under the psycholo...,Forbes,2021-01-02 00:20:00+00:00,"liquidity, volatility, price action, price mov...",0.238574,2021 Bitcoin Price Predictions: Is The Massive...,neutral,0.876394,0.5,0.0,0.0,0.0,0.0,0.0,[],[],[Bitcoin],[]
1,Will Central Banks Hold Bitcoin in 2021?,Central banks (CB) will hold bitcoin sooner or...,BeInCrypto,2021-01-01 20:31:35+00:00,consensus updates,0.199005,Will Central Banks Hold Bitcoin in 2021? Centr...,neutral,0.916949,0.0,0.5,0.0,0.0,0.7,0.0,[Krüger],[Central banks],[Bitcoin],[]
2,"Bitcoin Is Digital Social Justice, feat. Tyron...",The podcaster and CEO of Onramp Invest discuss...,Coindesk,2021-01-01 19:15:02+00:00,social platforms,0.299585,"Bitcoin Is Digital Social Justice, feat. Tyron...",neutral,0.903567,0.0,0.3,0.0,0.5,0.7,0.0,[Tyrone Ross],[Onramp Invest],[Bitcoin],[]


In [ ]:
# try another NER method
nlp = pipeline("ner")

In [ ]:
'''
method 1:

Choose a Base Model
General-purpose: bert-base-uncased, roberta-base
Domain-specific: ElKulako/cryptobert (for crypto), ProsusAI/finbert (finance), or modernbert?

Load NER dataset (e.g., CoNLL-2003, or any other crypo news dataset with NER annotations)
Tokenize and Align Labels
Fine-tune the Model
Evaluate the Model (metrics like precision, recall, and F1-score)
'''

In [ ]:
'''
method 2:
directly use the NER model from Hugging Face
Jean-Baptiste/roberta-large-ner-english

'''